In [1]:
from ortools.sat.python import cp_model
import random
from collections import defaultdict

In [2]:
from typing import List

class NursePatientMatcher:
    
    def __init__(self, num_patients, num_nurses, max_patients, patient_types, acuities, patient_times):
        self.p: int = num_patients
        self.n: int = num_nurses
        self.max_patients_per_nurse: int = max_patients
        self.patients_of_each_type: List[int] = patient_types
        self.patient_nurse_acuities: List[List[int]] = acuities
        self.patient_times: List[int] = patient_times

In [3]:
def create_variables(self):
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    patients_per_types = self.patients_of_each_type
    max_patients: int = self.max_patients_per_nurse
    self.x = {}
    self.patients_per_nurse = {}
    self.patient_types = {}
    self.penalty = model.NewIntVar(0,1,'penalty')
    for nurse in range(n):
        for patient in range(p):
            self.x[nurse, patient] = model.NewBoolVar(f'x[Nurse: {nurse}, Patient: {patient}]')
    
    for nurse in range(n):
        self.patients_per_nurse[nurse] = model.NewIntVar(0, max_patients, f'Patients for Nurse {nurse}')
    
    prev = 0
    t = 0
    for patient in range(p):
        num_patients_of_type = patients_per_types[t]
        if patient >= prev + num_patients_of_type:
            prev = prev + num_patients_of_type
            t += 1
        self.patient_types[patient] = t

    self.patients_per_time = {}
    for i in range(10):
        self.patients_per_time[i] = [patient for patient in range(p) if self.patient_times[patient] == i]


    self.nurse_sched = []
    for nurse in range(n):
        self.nurse_sched.append([model.NewIntVar(0, p, f'Hour {i} for nurse {nurse}') for i in range(10)])
            
        

# Register this method with the solver class
NursePatientMatcher.create_variables = create_variables

In [4]:
def bound_patients_per_nurse(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    max_patients: int = self.max_patients_per_nurse
    # min_patients: int = self.min_patients_per_nurse
    for nurse in range(n):
        model.Add(sum(x[nurse,patient] for patient in range(p)) <= max_patients)
        # model.Add(sum(x[nurse,patient] for patient in range(p)) >= min_patients)

# Register this method with the solver class
NursePatientMatcher.bound_patients_per_nurse = bound_patients_per_nurse

In [5]:
def one_nurse_per_patient(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    for patient in range(p):
        model.Add(sum(x[nurse,patient] for nurse in range(n)) == 1)


# Register this method with the solver class
NursePatientMatcher.one_nurse_per_patient = one_nurse_per_patient

In [6]:
def fill_nurse_schedule(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    nurse_sched = self.nurse_sched
    patients_per_time = self.patients_per_time

    for nurse in range(n):
        for i in range(10):
            model.Add(nurse_sched[nurse][i] == sum(x[nurse, patient] for patient in patients_per_time[i]))
            model.Add(nurse_sched[nurse][i] <= 1)

# Register this method with the solver class
NursePatientMatcher.fill_nurse_schedule = fill_nurse_schedule
    

In [7]:
def track_patients_per_nurse(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    p: int = self.p
    patients_per_nurse = self.patients_per_nurse
    for nurse in range(n):
        model.Add(sum(x[nurse,patient] for patient in range(p)) == patients_per_nurse[nurse])


# Register this method with the solver class
NursePatientMatcher.track_patients_per_nurse = track_patients_per_nurse

In [8]:
def at_most_one_double_shift(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    nurse_sched = self.nurse_sched
    for nurse in range(n):
        num_double_tracker = [model.NewBoolVar(f'Double shifts for nurse {nurse}') for _ in range(9)]
        for i in range(1, 10):
            model.Add((nurse_sched[nurse][i] + nurse_sched[nurse][i-1] == 2)).OnlyEnforceIf(num_double_tracker[i-1])
            model.Add((nurse_sched[nurse][i] + nurse_sched[nurse][i-1] < 2)).OnlyEnforceIf(num_double_tracker[i-1].Not())
        
        pairs = [[model.NewBoolVar(f'Shift pair {j}, {k} for nurse {nurse}') for k in range(j+1, 9)] for j in range(11)]

        for j in range(9):
            for k in range(j+1, 9):
                model.AddBoolOr([num_double_tracker[j].Not(), num_double_tracker[k].Not()]).OnlyEnforceIf(pairs[j][k-j-1])
                model.AddBoolAnd([num_double_tracker[j], num_double_tracker[k]]).OnlyEnforceIf(pairs[j][k-j-1].Not())

        flattened_pairs = []
        for pair in pairs:
            flattened_pairs.extend(pair)
        
        model.AddBoolAnd(flattened_pairs)
        
# Register this method with the solver class
NursePatientMatcher.at_most_one_double_shift = at_most_one_double_shift

In [9]:
def double_shift_penalty(self):
    x = self.x
    model: cp_model.CpModel = self.model
    n: int = self.n
    nurse_sched = self.nurse_sched
    self.penalty = model.NewIntVar(0, 10*n, 'penalty')
    num_double_tracker = [[model.NewBoolVar(f'Double shifts for nurse {nurse}') for _ in range(9)] for nurse in range(n)]
    for nurse in range(n):
        for i in range(1, 10):
            model.Add((nurse_sched[nurse][i] + nurse_sched[nurse][i-1] == 2)).OnlyEnforceIf(num_double_tracker[nurse][i-1])
            model.Add((nurse_sched[nurse][i] + nurse_sched[nurse][i-1] < 2)).OnlyEnforceIf(num_double_tracker[nurse][i-1].Not())
        
    model.Add(self.penalty == sum(sum(num_double_tracker[nurse][i] for i in range(9)) for nurse in range(n)))
        
# Register this method with the solver class
NursePatientMatcher.double_shift_penalty = double_shift_penalty

In [10]:
def minimize_objectives(self):
    x = self.x
    model: cp_model.CpModel = self.model
    patients_per_nurse = self.patients_per_nurse
    n: int = self.n
    p: int = self.p
    max_patients: int = self.max_patients_per_nurse
    acuities = self.patient_nurse_acuities
    penalty = self.penalty

    absolute_diff = [model.NewIntVar(0, max_patients, "Absolute difference between nurse n workload and average workload") for _ in range(n)]

    for nurse in range(n):
        difference = model.NewIntVar(-max_patients, max_patients, f'Difference between nurse {nurse} workload and average workload')
        model.Add(difference == patients_per_nurse[nurse] - int(p/n))
        model.AddAbsEquality(absolute_diff[nurse], difference)
    

    self.acuity_score = sum(acuities[nurse][self.patient_types[patient]] * self.x[nurse, patient] for nurse in range(n) for patient in range(p))

    acuity_ub = sum(max(acuities[nurse]) for nurse in range(n))
    acuity_lb = sum(min(acuities[nurse]) for nurse in range(n))

    M = acuity_ub - acuity_lb + 1

    model.Minimize(M*sum(absolute_diff) - self.acuity_score + M*penalty)
    

# Register this method with the solver class
NursePatientMatcher.minimize_objectives = minimize_objectives

In [11]:
def solve(self):
    self.model = cp_model.CpModel()
    self.solver = cp_model.CpSolver()
    self.create_variables()
    self.bound_patients_per_nurse()
    self.one_nurse_per_patient()
    self.fill_nurse_schedule()
    self.track_patients_per_nurse()
    self.double_shift_penalty()
    self.minimize_objectives()
    if self.solver.Solve(self.model) == cp_model.OPTIMAL:
        print('Solved!')
        print(f'Average Nurse Acuity Score: {self.solver.Value(self.acuity_score) / self.n}')
        for nurse in range(self.n):
            print(f'Patients for nurse {nurse}: {self.solver.Value(self.patients_per_nurse[nurse])}')
            for i in range(10):
                print(f'{self.solver.Value(self.nurse_sched[nurse][i])}', end=' ')
            print()
        
    else:   
        raise ValueError('Modeling error!')

# Register this method with the solver class
NursePatientMatcher.solve = solve

In [12]:
def read_data_and_solve(path):
    with open(path) as f:
        lines = f.readlines()
        num_nurses, num_patients, num_patient_types = lines[0].split(' ')
        num_nurses, num_patients, num_patient_types = int(num_nurses), int(num_patients), int(num_patient_types)

        min_patients, max_patients = lines[1].split(' ')
        min_patients, max_patients = int(min_patients), int(max_patients)

        patient_types = lines[3].strip().split(' ')
        patient_types = [int(x) for x in patient_types]

        acuities = []
        for i in range(num_nurses):
            nurse_acuities = lines[5 + i].strip().split(' ')
            acuities.append([int(x) for x in nurse_acuities])

    patient_times = []
    times_seen = defaultdict(int)
    for _ in range(num_patients):
        new_time = random.randint(0,9)
        while not (not times_seen or times_seen[new_time] < num_nurses):
            new_time = random.randint(0,9)
        times_seen[new_time] += 1
        patient_times.append(new_time)

    matcher = NursePatientMatcher(num_patients, num_nurses, max_patients, patient_types, acuities, patient_times)
    soln = matcher.solve()

In [13]:
read_data_and_solve('data/hospital_data.txt')

Solved!
Average Nurse Acuity Score: 231.32
Patients for nurse 0: 7
1 0 1 1 1 0 1 1 0 1 
Patients for nurse 1: 7
1 0 1 0 1 1 0 1 1 1 
Patients for nurse 2: 7
1 1 0 1 0 1 1 0 1 1 
Patients for nurse 3: 7
1 0 1 0 1 1 0 1 1 1 
Patients for nurse 4: 7
1 0 1 0 1 1 0 1 1 1 
Patients for nurse 5: 7
1 1 1 1 1 0 1 0 1 0 
Patients for nurse 6: 7
1 1 0 1 1 0 1 0 1 1 
Patients for nurse 7: 7
1 0 1 0 1 0 1 1 1 1 
Patients for nurse 8: 7
1 0 1 1 1 1 0 1 0 1 
Patients for nurse 9: 7
1 1 1 1 1 0 1 0 1 0 
Patients for nurse 10: 7
1 0 1 1 1 1 0 1 1 0 
Patients for nurse 11: 7
1 0 1 0 1 1 1 0 1 1 
Patients for nurse 12: 7
0 1 1 1 1 1 0 1 1 0 
Patients for nurse 13: 7
0 1 1 1 1 0 1 0 1 1 
Patients for nurse 14: 7
1 1 1 1 0 1 0 1 0 1 
Patients for nurse 15: 7
1 1 0 1 0 1 0 1 1 1 
Patients for nurse 16: 7
1 1 1 0 1 0 1 0 1 1 
Patients for nurse 17: 7
0 1 1 1 1 1 0 1 0 1 
Patients for nurse 18: 7
0 1 1 1 0 1 0 1 1 1 
Patients for nurse 19: 7
1 1 0 1 1 0 1 0 1 1 
Patients for nurse 20: 7
1 1 1 1 1 0 1 0 1 0 
P